Remaining TODO:

1. Add titles
2. Determine correct `zoom` values for each state
3. Run GIFmaker for all

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import config
import imageio

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from PIL import Image, ImageFont, ImageDraw

In [2]:
mapbox_access_token = config.mapbox_access_token
plotly.tools.set_credentials_file(username=config.plotly_username, api_key=config.plotly_api_key)

In [3]:
ge = pd.read_csv('data/election-data-2004-2018.csv')

Same changes as in `explorer.ipynb`.

In [4]:
ge.at[849, 'winlose'] = 0
ge['coalition'].fillna('Unaffiliated', inplace=True)

In [5]:
years = ge['year'].unique()
coals = ge['coalition'].unique()
states = ge['state'].unique()

In [6]:
coals

array(['BN', 'Pakatan', 'Unaffiliated', 'PAS'], dtype=object)

In [7]:
coal_colors = dict(
    BN='rgba(9, 39, 129, 0.5)', 
    Pakatan='rgba(237, 28, 36, 0.5)', 
    PAS='rgba(0, 136, 0, 0.5)',
    Unaffiliated='rgba(26, 26, 26, 0.5)'
)

In [8]:
def map_data_layout(year, area):    
    lat = geo[area]['lat']
    lon = geo[area]['lon']
    key = 'malaysia' if (area is 'west_malaysia') else str(area)
    zoom = geo[area]['zoom']
    
    data = go.Data([go.Scattermapbox(lat=lat, lon=lon, mode='markers')])
    layout = go.Layout(
        height=1100, width=1100, autosize=False, hovermode=False,
        margin=go.Margin(l=0, r=0, b=0, t=0, pad=0),
        mapbox=dict(
            layers=[{
                'source': f'https://raw.githubusercontent.com/theianchan/ge14-map/master/data/ge-{str(year)}-{coal.lower()}-{key}.geojson',
                'sourcetype': 'geojson',
                'type': 'fill',
                'color': coal_colors[coal]
            } for coal in coals],
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(lat=lat, lon=lon),
            zoom=zoom,
            style='light',
            pitch=0
        )
    )
    return data, layout

In [17]:
py.iplot(get_map(2018, 'malaysia'), filename='ge-2018-malaysia')

PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per day. If you need to raise your daily limit, consider upgrading to a Student or Personal Plan
see: https://plot.ly/products/cloud

In [9]:
def get_map(year, area, display=True, save=True):
    data, layout = map_data_layout(year, area)
    if save:
        fname = f'maps/ge-{str(year)}-{str(area)}.png'
        py.image.save_as(go.Figure(data=data, layout=layout), filename=fname)
        print(f'Saved {fname}.')
    if display:
        return dict(data=data, layout=layout)

In [10]:
geo = dict(
    malaysia=dict(
        lat=4.127928, lon=110.120693, zoom=4.8),
    west_malaysia=dict(
        lat=4.127928, lon=101.709438, zoom=6.0),
#     perlis=dict(
#         lat=6.490206, lon=100.105001, zoom=2.0),
#     kedah=dict(
#         lat=5.810463, lon=99.8142927, zoom=2.0),
#     kelantan = dict(
#         lat=5.395310, lon=101.439472, zoom=2.0),
#     terengganu = dict(
#         lat=4.922274, lon=102.474374, zoom=2.0),
#     penang = dict(
#         lat=5.353988, lon=100.222916, zoom=2.0),
#     perak = dict(
#         lat=4.800220, lon=99.9356662, zoom=2.0),
#     pahang = dict(
#         lat=3.618645, lon=101.656548, zoom=2.0),
#     selangor = dict(
#         lat=3.232068, lon=100.820492, zoom=2.0),
#     wp_kuala_lumpur = dict(
#         lat=3.138503, lon=101.616949, zoom=2.0),
#     wp_putrajaya = dict(
#         lat=2.926361, lon=101.661425, zoom=2.0),
#     negeri_sembilan = dict(
#         lat=2.839789, lon=101.921915, zoom=2.0),
#     melaka = dict(
#         lat=2.237377, lon=102.181463, zoom=2.0),
#     johor = dict(
#         lat=2.049175, lon=102.949493, zoom=2.0),
#     wp_labuan = dict(
#         lat=5.316768, lon=115.142799, zoom=2.0),
#     sabah = dict(
#         lat=5.738876, lon=116.183216, zoom=2.0),
#     sarawak = dict(
#         lat=2.915459, lon=110.363620, zoom=2.0)
)

# Get correct zoom

In [ ]:
py.iplot(get_map(2018, 'malaysia'), filename='ge-2018-malaysia')

In [ ]:
py.iplot(get_map(2018, 'perlis'), filename='ge-2018-perlis')

# Create maps

In [ ]:
for g in geo:
    for year in years:
        get_map(year, g, display=False)

## Process images and create GIFs

In [65]:
font_heading = ImageFont.truetype('fonts/Lato-Bold.ttf', 60)
font_paragraph = ImageFont.truetype('fonts/Lato-Regular.ttf', 24)
font_color_dark = (34, 34, 34)

In [66]:
x_lm = 40
y_tm = 40
y_cp = 40 + 84

y_bn = 904
y_ph = 904 + 42
y_gs = 904 + 42 + 42
y_ot = 904 + 42 + 42 + 42

x_ch = 287
h_ch = 30

c_bn = (9, 39, 129)
c_ph = (237, 28, 36)
c_gs = (0, 136, 0)
c_ot = (26, 26, 26)

In [67]:
def num_seats_won(year, coalition):
    return ge[(ge['year'] == year) & (ge['coalition'] == coalition)]['winlose'].sum()

In [68]:
for g in geo:
    for year in years:
        img = Image.open('maps/preview-blank.png')
        draw = ImageDraw.Draw(img)
        draw.text((x_lm, y_tm), 'GE 14 (WP Wilayah Persekutuan)', font_color_dark, font_heading)
        draw.text((x_lm, y_cp), 'Created for datatarik.com', font_color_dark, font_paragraph)

        seats = num_seats_won(year, 'BN')
        draw.text((x_lm, y_bn), 'Barisan National', font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_bn), (x_ch + seats * 3, y_bn + h_ch)], fill=c_bn)
        draw.text((x_ch + seats * 3 + 12, y_bn), str(seats), font_color_dark, font_paragraph)

        seats = num_seats_won(year, 'Pakatan')
        draw.text((x_lm, y_ph), 'Pakatan Harapan', font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_ph), (x_ch + seats * 3, y_ph + h_ch)], fill=c_ph)
        draw.text((x_ch + seats * 3 + 12, y_ph), str(seats), font_color_dark, font_paragraph)

        seats = num_seats_won(year, 'PAS')
        draw.text((x_lm, y_gs), 'Gagasan Sejahtera', font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_gs), (x_ch + seats * 3, y_gs + h_ch)], fill=c_gs)
        draw.text((x_ch + seats * 3 + 12, y_gs), str(seats), font_color_dark, font_paragraph)

        seats = num_seats_won(year, 'Unaffiliated')
        draw.text((x_lm, y_ot), 'Unaffiliated (Other)', font_color_dark, font_paragraph)
        draw.rectangle([(x_ch, y_ot), (x_ch + seats * 3, y_ot + h_ch)], fill=c_ot)
        draw.text((x_ch + seats * 3 + 12, y_ot), str(seats), font_color_dark, font_paragraph)

        fname = f'frames/ge-{str(year)}-{g}.png'
        img.save(fname)
        print(f'Saved {fname}.')

Saved frames/ge-2004-malaysia.png.
Saved frames/ge-2008-malaysia.png.
Saved frames/ge-2013-malaysia.png.
Saved frames/ge-2018-malaysia.png.
Saved frames/ge-2004-west_malaysia.png.
Saved frames/ge-2008-west_malaysia.png.
Saved frames/ge-2013-west_malaysia.png.
Saved frames/ge-2018-west_malaysia.png.


In [69]:
for g in geo:
    frames = []
    for year in years:
        frames.append(imageio.imread(f'frames/ge-{str(year)}-{str(g)}.png'))
    fname = f'gifs/ge-{str(g)}.gif'
    imageio.mimwrite(fname, frames, 'GIF', duration=2)
    print(f'Saved {fname}.')

Saved gifs/ge-malaysia.gif.
Saved gifs/ge-west_malaysia.gif.
